# Scheduled trains maximization with GSA

## 0. Load libraries

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from src.entities import GSA

from typing import Union

## 1. Define corridor

As an example, we will use the Spanish south high-speed railway corridor.



In [2]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                        },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

### 1.1. Get lines from corridor

In [3]:
def get_lines(corridor: dict, path: Union[list, None]=None) -> list:
    """
    Get all the lines in the corridor
    
    Args:
        corridor (dict): dictionary with the corridor structure
        path (list, optional): list of nodes
    
    Returns:
        list of lines
    """
    if path is None:
        path = []

    lines = []
    for node, child in corridor.items():
        new_path = path + [node]
        if not child:  # If the node has no children, it is a leaf
            lines.append(new_path)
        else:
            lines.extend(get_lines(child, new_path))  # If the node has children, we call the function recursively

    return lines

get_lines(corridor)

[['MAD', 'CIU', 'COR', 'SEV', 'CAD'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']]

### 1.2. Sample a random line, and a random route from the line

In [4]:
def sample_line(lines: list) -> list:
    """
    Sample a random line from the list of lines
    
    Args:
        lines (list): list of lines
    
    Returns:
        list: random line
    """
    return lines[np.random.randint(len(lines))] 

def sample_route(line: list) -> list:
    """
    Sample a random route from line. At least two stations.
    
    Args:
        line (list): list of stations
        
    Returns:
        list: random route
    """
    return line[np.random.randint(0, len(line)-1):]

lines = get_lines(corridor)

line = sample_line(lines)
print(f"Sampled line: {line}")

# Sample a random route from line (at least two stations)
route = sample_route(line)
print(f"Sampled route: {route}")

Sampled line: ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']
Sampled route: ['COR', 'PGE', 'ANT', 'MAL']


### 1.3 Generate random timetable for a route

- Times in minutes.
- Initial time is randomized between 0 and 24*60 (24 hours).
- The time between stations is also randomized between 30 and 120 minutes.
- The time to dwell at each station is randomized between 2 and 8 minutes.

In [5]:
def get_timetable(route: list) -> dict:
    """
    Generate random timetable for route r
    
    Args:
        route (list): list of stations
    
    Returns:
        dict: timetable
    """
    timetable = {}
    AT = np.random.randint(0, 24*60)
    DT = AT
    for i, sta in enumerate(route):
        if i == 0 or i == len(route)-1:
            timetable[sta] = (AT, AT)
        else:
            timetable[sta] = (AT, DT)
            
        AT += np.random.randint(30, 120)
        DT = AT + np.random.randint(2, 8)
        
    return timetable

get_timetable(route)

{'COR': (168, 168), 'PGE': (283, 287), 'ANT': (376, 379), 'MAL': (423, 423)}

## 2. Generate as many requested services as needed

In [7]:
# Generate random requested timetable in corridor for a day t

# Number of services:
n_services = 200

# 1) Generate random timetable
schedule = {i: get_timetable(sample_route(sample_line(lines))) for i in range(1, n_services+1)}
schedule

{1: {'MAD': (1429, 1429),
  'CIU': (1506, 1509),
  'COR': (1622, 1624),
  'SEV': (1674, 1676),
  'CAD': (1709, 1709)},
 2: {'CIU': (841, 841),
  'COR': (903, 906),
  'PGE': (974, 976),
  'ANT': (1025, 1028),
  'GRA': (1125, 1125)},
 3: {'MAD': (220, 220),
  'CIU': (329, 332),
  'COR': (391, 393),
  'PGE': (496, 501),
  'ANT': (566, 568),
  'MAL': (638, 638)},
 4: {'MAD': (825, 825),
  'CIU': (867, 870),
  'COR': (951, 957),
  'SEV': (1002, 1009),
  'CAD': (1037, 1037)},
 5: {'MAD': (1176, 1176),
  'CIU': (1207, 1214),
  'COR': (1301, 1308),
  'PGE': (1407, 1410),
  'ANT': (1468, 1473),
  'GRA': (1576, 1576)},
 6: {'COR': (1220, 1220),
  'PGE': (1335, 1337),
  'ANT': (1435, 1439),
  'GRA': (1535, 1535)},
 7: {'MAD': (95, 95),
  'CIU': (170, 173),
  'COR': (244, 250),
  'SEV': (295, 301),
  'CAD': (330, 330)},
 8: {'COR': (1047, 1047), 'SEV': (1106, 1113), 'CAD': (1208, 1208)},
 9: {'PGE': (533, 533), 'ANT': (619, 623), 'GRA': (729, 729)},
 10: {'SEV': (1352, 1352), 'CAD': (1408, 1408)},

## 3. Define feasibility function

A service $ i $ is feasible (can be scheduled and, therefore, $ S_i $ take the value 1) if the following condition is met:

$ | DT_{ij} - DT_{kj} | \geq security\_gap $

Where:
- $ DT_{ij} $ is the departure time for service $ i $ from station $ j $.
- $  DT_{kj} $ is the departure time for service $ k $ from the same station $ j $.

Inequality must be met for all services ($ k \neq j $) that also operate at station $ j $ for service $ i $ to be feasible.

1) Requests from the Railway Undertakings (RUs) are considered as a set of services that can be scheduled or not. The decision variable $ S_i $ is binary and takes the value 1 if the service $ i $ is scheduled and 0 otherwise.

Service ID |  RU   |          Trip          |              Requested schedule              | 
:---: |:-----:|:----------------------:|:--------------------------------------------:|
1 | Renfe |   `[MAD, BAR, FIG]`    |      `[(0, 0), (148, 152), (180, 180)]`      |
2 | Ouigo | `[MAD, ZAR, BAR, FIG]` | `[(8, 8), (28, 30), (165, 167), (210, 210)]` |
3 | Iryo  |   `[MAD, BAR, FIG]`    |     `[(30, 30), (180, 182), (225, 225)]`     |


2) Get conflict matrices for each service. A conflict matrix is a binary matrix of 0's and 1's. A 1 appears in the outputs of services (different from the one we are analyzing) that interferes with a station of the service in question.

Service 1: Has an interference with the service 2 at the departure from Madrid.

Service ID | MAD | BAR | FIG |
:---: | :---: | :---: | :---: |
1 | 0 | 0 | 0 |
2 | 1 | 0 | 0|
3 | 0 | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_1} = (S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) = 0 $ \\

Service 2: Has an interference with the service 1 at the departure from Madrid.

ID servicio | MAD | ZAR | BAR | FIG |
:---: | :---: | :---: | :---: | :---: |
1 | 0 | 0 | 0 | 0 |
2 | 1 | 0 | 0 | 0|
3 | 0 | 0 | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_2} = (S_1 \cdot 1 + S_1 \cdot 0 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) = 0 $ \\

Service 3: Does not have any interference with the other services.

ID servicio | MAD | BAR | FIG |
:---: |:---:| :---: | :---: |
1 |  0  | 0 | 0 |
2 |  0  | 0 | 0|
3 |  0  | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_2} = (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) \leq 0 $ \\

We can define a general restriction to check if the timetable is feasible or not:

$ S_1 \cdot R_{S_1} + S_2 \cdot R_{S_2} + S_2 \cdot R_{S_2} \leq 0$

This equation can be read as follows:

- If a service $ S_i $ is planned, then its restrictions $ R_{S_i} $ must be met. If the service $ S_i $ is not planned, its own interferences do not have to be taken into account.
- For the set of schedules to be feasible, there must be no conflict between any of the planned services (all the $ S_i $ that take the value $ 1 $)

In [8]:
def is_feasible(solution, security_gap=10):
    """
    Check if the solution is feasible
    
    Args:
        solution (dict): solution
        security_gap (int, optional): security gap
        
    Returns:
        bool: True if the solution is feasible, False otherwise
    """
    solution = np.array(solution["discrete"])
    security_array = []
    
    # Get conflicts between services
    for service in schedule:
        service_sec_arr = []  # Build security matrix for service (columns are stops, rows are other services)
        for service_k in schedule:
          service_sec_row = [] 
          for stop in schedule[service]:
            if service_k == service or stop not in schedule[service_k]:
              service_sec_row.append(0)
              continue
        
            if abs(schedule[service][stop][1] - schedule[service_k][stop][1]) < security_gap:
              service_sec_row.append(1)
            else:
              service_sec_row.append(0)
          service_sec_arr.append(service_sec_row)
        
        security_array.append(np.array(service_sec_arr))
    
    # Get array of security conflicts. If there is a conflict, the dot product will be different from zero
    if not solution.dot(np.array([np.sum(solution.dot(ssa)) for ssa in security_array])):
        return True
    return False

"""
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

solution = {"discrete": [1, 0, 1]}

is_feasible(solution)
"""

'\nschedule = {1: {\'MAD\': (0, 0), \'BAR\': (148, 152), \'FIG\': (180, 180)},\n            2: {\'MAD\': (8, 8), \'ZAR\': (28, 30), \'BAR\': (165, 167), \'FIG\': (210, 210)},\n            3: {\'MAD\': (30, 30), \'BAR\': (180, 182), \'FIG\': (225, 225)}}\n\nsolution = {"discrete": [1, 0, 1]}\n\nis_feasible(solution)\n'

# 4. Define objective function

In [9]:
def get_num_trains(solution):
    scheduled_trains = np.sum(solution["discrete"])
    print("Requested number of trains: ", len(solution["discrete"]))
    print("Scheduled trains: ", scheduled_trains)
    return scheduled_trains, 0

# 5. Optimize with GSA

In [12]:
boundaries = {'real': [], 'discrete': [(0, 1) for _ in range(len(schedule))]}

gsa_algo = GSA(objective_function=get_num_trains,
               is_feasible=is_feasible,
               r_dim=0,
               d_dim=len(schedule),
               boundaries=boundaries)

In [13]:
gsa_algo.set_seed(seed=28)

training_history = gsa_algo.optimize(population_size=5,
                                     iters=10,
                                     chaotic_constant=False,
                                     repair_solution=False)

Initializing positions of the individuals in the population...
Positions of the individuals in the population successfully initialized!!
GSA is optimizing  "get_num_trains"
Requested number of trains:  200
Scheduled trains:  15
Requested number of trains:  200
Scheduled trains:  13
Requested number of trains:  200
Scheduled trains:  6
Requested number of trains:  200
Scheduled trains:  13
Requested number of trains:  200
Scheduled trains:  8
['At iteration 1 the best fitness is 15']
Requested number of trains:  200
Scheduled trains:  23
Requested number of trains:  200
Scheduled trains:  18
Requested number of trains:  200
Scheduled trains:  34
Requested number of trains:  200
Scheduled trains:  20
Requested number of trains:  200
Scheduled trains:  33
['At iteration 2 the best fitness is 34']
Requested number of trains:  200
Scheduled trains:  26
Requested number of trains:  200
Scheduled trains:  34
Requested number of trains:  200
Scheduled trains:  31
Requested number of trains:  2